<a href="https://colab.research.google.com/github/kharlescim/ERT/blob/main/ERT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Colab Purpose**
This colab will mainly be used for documenting my thoughts and manipulating the datasets using xarray, as it's easier to quickly visualize and print information from the datasets in this environment. However, training the model on colab takes wayyy too much RAM, enough to crash after just one epoch. As such, training will be done on a different .py file in the repo, although the code will still be written here. For now, all my thoughts and documentation, including on the training done from my machine, will be here. the .py code is just to run the model.

**Loading and printing datasets from nc files**

In [ ]:
!pip install xarray netCDF4

import xarray as xr
import numpy as np
import pandas as pd

In [ ]:
LTD_ds = xr.open_dataset('LTD05.nc')
USDM05_ds = xr.open_dataset('USDM05_2000_2024.nc')
obs_ds = xr.open_dataset('obs.nc')
spei_ds = xr.open_dataset('spei_obs_3D.nc')


# LTD and USDM only have one variable, so they can be treated as DataArrays
LTD = LTD_ds['LTD']
USDM = USDM05_ds['USDM']
print(LTD)
print(USDM)
print(obs_ds)
print(spei_ds)



<xarray.DataArray 'LTD' (time: 1305, lat: 48, lon: 118)> Size: 30MB
[7391520 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 472B -125.8 -125.2 -124.8 ... -68.25 -67.75 -67.25
  * lat      (lat) float32 192B 25.25 25.75 26.25 26.75 ... 47.75 48.25 48.75
  * time     (time) datetime64[ns] 10kB 2000-01-04 2000-01-11 ... 2024-12-31
<xarray.DataArray 'USDM' (time: 1305, lat: 48, lon: 118)> Size: 30MB
[7391520 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 472B -125.8 -125.2 -124.8 ... -68.25 -67.75 -67.25
  * lat      (lat) float32 192B 25.25 25.75 26.25 26.75 ... 47.75 48.25 48.75
  * time     (time) datetime64[ns] 10kB 2000-01-04 2000-01-11 ... 2024-12-31
<xarray.Dataset> Size: 193MB
Dimensions:  (lon: 118, lat: 48, ens: 1, time: 406)
Coordinates:
  * lon      (lon) float32 472B -125.8 -125.2 -124.8 ... -68.25 -67.75 -67.25
  * lat      (lat) float32 192B 25.25 25.75 26.25 26.75 ... 47.75 48.25 48.75
  * ens      (ens) float32 4B 1.0
  * time     (t

**General Knowledge** <br>
- LTD = Long Term Drought (already in USDM format)
- USDM = USDM Rasterdized Data
- obs = VIC Model Drought Index
- spei = SPEI Drought Index

LTD & USDM have weekly temporal frequency, obs and spei are monthly. also obs and spei have multiple variables and should be treated as datasets, while LTD and USDM can simply be treated as dataarrays

**Data Cleanup TODO** <br>
interpolate OBS and SPEI data into monthly data (method = linear)<br>
standardize all datasets using USDM categorical distribution <br>
see how plot animation changes between tasks


**Interpolating Data**

In [ ]:
# Checking date ranges
print(obs_ds.time.values[0])
print(obs_ds.time.values[-1])
print(spei_ds.time.values[0])
print(spei_ds.time.values[-1])

1991-01-01T00:00:00.000000000
2024-10-01T00:00:00.000000000
1991-01-31T00:00:00.000000000
2024-08-31T00:00:00.000000000


In [ ]:
# Creating new time coordinate
weekly_time = pd.date_range(
    start=obs_ds.time.values[0],
    end=obs_ds.time.values[-1],
    freq='W'  # weekly frequency
)

# Converting to weekly (method = linear)
obs_weekly = obs_ds.interp(time=weekly_time, method="linear")
spei_weekly = spei_ds.interp(time=weekly_time, method="linear")



**Converting to Categorical USDM using Percentiles**
- CONSIDER DOING - add another category for NaN/non-drought values, so we can better visualize drought vs non-drought areas on map
  - when visualizing USDM-converted datasets, the border of map also completely disappears

In [ ]:
# Function to convert raw values to percentiles
def to_percentile(ds, dim='time'):
    # Convert each grid point's time series to percentile values.
    return ds.rank(dim=dim, pct=True)

# Function to bucket entries w/ specific percentiles in matching usdm categories
def percentile_to_USDM(p):
  return xr.where(p <= 0.02, 4,           # D4 (Exceptional Drought) = .00-.02
           xr.where(p <= 0.05, 3,         # D3 (Extreme Drought) = .02-.05
           xr.where(p <= 0.10, 2,         # D2 (Severe Drought) = .05-.10
           xr.where(p <= 0.20, 1, 0))))   # D1 (Moderate Drought) = .1-.2, else D0 (Abnormally Dry)


percentiles_obs = to_percentile(obs_weekly)
USDM_obs = percentile_to_USDM(percentiles_obs)
percentiles_spei = to_percentile(spei_weekly)
USDM_spei = percentile_to_USDM(percentiles_spei)

quick verification

In [ ]:
# print(USDM_obs)
# print(USDM_spei)
print(USDM_obs.to_array().mean().item())
print((USDM_obs.to_array() > 0).sum().item())
print(USDM_spei.to_array().mean().item())
print((USDM_spei.to_array() > 0).sum().item())

**Visualizing Datasets w/ Animations**

to be used to animate USDM datasets (for now, just visualizing LTD and USDM)

In [ ]:
from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import BoundaryNorm
import matplotlib.cm as cm

fig, ax = plt.subplots()

# Discrete colormap and norm for categories 0–4
bounds = [0, 1, 2, 3, 4, 5]
cmap = cm.get_cmap("magma", len(bounds) - 1)
norm = BoundaryNorm(bounds, cmap.N)

cbar = None

# Define the update function
def animate_USDM(data, i):
    global cbar
    ax.clear()

    # Store the plot object
    # Plot with discrete colormap and norm
    plot_obj = data.isel(time=i).plot(
        ax=ax,
        cmap=cmap,
        norm=norm,
        add_colorbar=False
    )

    # Add static colorbar once
    if cbar is None:
        cbar = fig.colorbar(
            plot_obj,
            ax=ax,
            boundaries=bounds,
            ticks=range(5),
            label='USDM Category'
        )
        cbar.ax.set_yticklabels(['D0', 'D1', 'D2', 'D3', 'D4'])

    ax.set_title(str(data.time[i].values)[:10])
    ax.set_title(str(data.time[i].values)[:10])
    # Return the plot object
    return plot_obj

In [ ]:
# plotting raw LTD dataset (LTD is already in USDM range)
anim_LTD = animation.FuncAnimation(fig, lambda i: animate_USDM(LTD, i), frames=len(LTD.time), interval=100)
HTML(anim_LTD.to_html5_video())

In [ ]:
# plotting raw USDM dataset
anim_USDM = animation.FuncAnimation(fig, lambda i: animate_USDM(USDM, i), frames=len(USDM.time), interval=100)
HTML(anim_USDM.to_html5_video())

**Training an MI Model using Keras** <br>
For now, we'll train the model following the basic examples using the utilities given by Keras, such as FeatureSpace. Later if needed, we can try implementing the model from scratch to better manage parameters

Possible examples from Keras: <br>
- **Structured Data Classification using FeatureSpace**
  - predict USDM drought class from lat/lon/time
- Timeseries Classification (pdf notes that these are potentially valuable, but weren't particularly studied)
  - predict future LTD values from past LTD datasets/values




importing relevant keras stuff

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
import keras
from keras.utils import FeatureSpace

converting and cleaning LTD to dataframe <br>
**Important:** changing datetime to Unix epoch format for use with TensorFlow
- may want to later extract month + day values. for now this is what we got

also changing LTD to be int

In [ ]:
LTD_df = LTD.to_dataframe().reset_index()
print(np.sort(LTD_df['LTD'].unique()))

# cleaning up NaN entries and -1 entries
df = LTD_df.dropna(subset=['LTD'])
df = df[df['LTD'] != -1.0]

#visualizing
print(df['LTD'].value_counts().sort_index())


# changing datetime to be usable by TensorFlow, changing LTD from float to int to match example
df["time"] = pd.to_datetime(df["time"]).map(pd.Timestamp.timestamp)
df["LTD"] = df["LTD"].astype(int)
df.head() # LTD is target

[-1.  0.  1.  2.  3.  4. nan]
LTD
0.0    314663
1.0    310012
2.0    338081
3.0    216478
4.0     82860
Name: count, dtype: int64


,time,lat,lon,LTD
147554,962668800.0,26.25,-98.75,1
147555,962668800.0,26.25,-98.25,1
147556,962668800.0,26.25,-97.75,1
147557,962668800.0,26.25,-97.25,1
147671,962668800.0,26.75,-99.25,1


**Applying Keras Structured Data Classification w/ Feature Space to cleaned LTD dataframe**

splitting data into training + validation set

In [ ]:
val_df = df.sample(frac=0.2, random_state=1337)
train_df = df.drop(val_df.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_df), len(val_df))
)

Using 1009675 samples for training and 252419 for validation


Generating tf.data.Dataset for each dataframe, where each Dataset yields a tuple (input, target). <br>
Input is a dictionary of features, target (LTD) is a value 0-4 <br>

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("LTD")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_df)
val_ds = dataframe_to_dataset(val_df)

for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'time': <tf.Tensor: shape=(), dtype=float64, numpy=1041292800.0>, 'lat': <tf.Tensor: shape=(), dtype=float32, numpy=36.25>, 'lon': <tf.Tensor: shape=(), dtype=float32, numpy=-111.75>}
Target: tf.Tensor(4, shape=(), dtype=int64)


Batching Datasets

In [ ]:
train_ds = train_ds.batch(16)
val_ds = val_ds.batch(16)

Configuring our FeatureSpace
- for now, features are pretty barebones. just as they are in original dataframe

In [ ]:
feature_space = FeatureSpace(
    features={
        "lat": "float",
        "lon": "float",
        "time": "float",
    },
    output_mode="concat"
)

Adapting FeatureSpace to training data

In [ ]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

preprocessed_x.shape: (16, 3)
preprocessed_x.dtype: <dtype: 'float32'>


Manage preprocessing in tf.data pipeline

In [ ]:
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

Build training & Inference Model

In [ ]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(32, activation="relu")(encoded_features)
x = keras.layers.Dropout(0.5)(x)
# change from example in order to fit categorization of LTD
predictions = keras.layers.Dense(5, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"] # change this too to from binary
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)

Train Model for 20 Epochs

From tests on my own machine, accuracy plateaus at 0.2678 at the second or third epoch, indicating that while the model is more accurate than pure random (1/5 chance), it suffers from underfitting/not being able to discern enough information well enough to predict correctly past a certain threshold. While this could be attributed to how the model was trained, I think it's more likely because it was only trained on the features given by LTD, which were lat, lon, and timezone (converted into Unix Epoch to account for TensorFlow) - not really good indicators for when a drought would come. I think the model would need to be trained on the other .nc files to be more accurate, but it could still be an issue with how I trained the model. need to ask.

In [ ]:
training_model.fit(
    preprocessed_train_ds,
    epochs=20,
    validation_data=preprocessed_val_ds,
    verbose=2,
)

Epoch 1/20
63105/63105 - 135s - 2ms/step - accuracy: 0.2457 - loss: -1.0368e+13 - val_accuracy: 0.2456 - val_loss: -2.9856e+13
Epoch 2/20
63105/63105 - 139s - 2ms/step - accuracy: 0.2456 - loss: -6.7129e+13 - val_accuracy: 0.2456 - val_loss: -1.1327e+14
Epoch 3/20
63105/63105 - 149s - 2ms/step - accuracy: 0.2456 - loss: -1.7696e+14 - val_accuracy: 0.2456 - val_loss: -2.4970e+14
Epoch 4/20


KeyboardInterrupt: 